In [161]:
import pandas as pd
import tqdm

import handler
import sqlalchemy as sql
from alchemy_utils import Base

session = handler.Session()

In [102]:
all_departed = """
SELECT *
FROM leg
WHERE departure_date BETWEEN date_add('2018-01-15 06:45:00', INTERVAL 30 day) AND SUBDATE(NOW(), 1)
"""

with handler.engine.connect() as con:
    df = pd.read_sql_query(all_departed, con)

In [128]:
# the combination of these columns forms a natural key
primary_cols = ['departure_location', 'arrival_location', 'departure_date', 'airline']
flights = [x for x in df.groupby(primary_cols)]

In [169]:
flight = min(flights, key=lambda x: len(x[1]))[0]

In [162]:
min_data_points = 30
f = {}
for key, d in tqdm.tqdm(flights):
    if len(d) < min_data_points:
        continue
    d['time_until_departure'] = d['departure_date'] - d['request_time']
    d['indexed_duration'] = d['duration'] - pd.Timestamp('1970-01-01 00:00:00')
    d = d.drop(labels=primary_cols + ['request_time', 'duration'], axis=1).set_index('id')
    f[key] = d

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7617/7617 [10:56:31<00:00,  5.17s/it]


In [164]:
import pickle
import os

In [175]:
with open(os.path.join(os.pardir, 'data', 'large_data', 'price_against_request_time.pkl'), 'wb') as file:
    pickle.dump(f, file)

In [172]:
test = [x for x in f.keys()][0]

6999